In [1]:
import panflute
panflute.__spec__

ModuleSpec(name='panflute', loader=<_frozen_importlib_external.SourceFileLoader object at 0x101fd1d68>, origin='/usr/local/lib/python3.5/site-packages/panflute/__init__.py', submodule_search_locations=['/usr/local/lib/python3.5/site-packages/panflute'])

In [2]:
%%writefile test-simple.md
~~~csv
table-width: 1.2
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~


Overwriting test-simple.md


In [3]:
%%writefile test-full.md
~~~csv
title: "*Great* Title"
has-header: False
column-width:
  - 0.1
  - 0.2
  - 0.3
  - 0.4
alignment: LRC
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~


Overwriting test-full.md


In [4]:
%%writefile test-include.md
~~~csv
title: "*Great* Title"
has-header: False
column-width: 0.1, 0.2, 0.3, 0.4
alignment: AlignLeft, AlignRight, AlignCenter, AlignDefault
source: ../source/grid.csv
~~~

Overwriting test-include.md


In [5]:
%%writefile ../bin/csv-tables.py
#!/usr/bin/env python3

"""
Panflute filter to parse CSV in fenced YAML code blocks
e.g.

```markdown
~~~csv
title: "*Great* Title"
has-header: False
column-width:
  - 0.1
  - 0.2
  - 0.3
  - 0.4
alignment: LRC
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~
```
"""

import io
import csv
import panflute

def fenced_csv(options, data, element, doc):
    # read csv and convert to panflute table representation
    with io.StringIO(data) as f:
        reader = list(csv.reader(f))
        body = []
        for row in reader:
            cells = [panflute.TableCell(*panflute.convert_text(x)) for x in row]
            body.append(panflute.TableRow(*cells))
        # get no of columns for header
        noOfColumn = len(reader[0])
    # read YAML metadata
    try:
        caption = options.get('title')
        column_width = options.get('column-width')
        table_width = options.get('table-width',1.0)
        alignment = options.get('alignment')
        has_header = options.get('has-header',True)
    except AttributeError:
        caption = None
        column_width = None
        table_width = 1.0
        alignment = None
        has_header = True
    # get caption
    if caption != None:
        caption = panflute.convert_text(caption)[0].content
    # get column_width
    if column_width == None:
        column_width_abs = [max([max([len(line) for line in row[i].split("\n")]) for row in list(reader)]) for i in range(noOfColumn)]
        column_width_tot = sum(column_width_abs)
        column_width = [column_width_abs[i]/column_width_tot*float(table_width) for i in range(noOfColumn)]
    # get alignment
    if alignment != None:
        parsed_alignment = []
        for i in range(noOfColumn):
            try:
                if alignment[i].lower() == "l":
                    parsed_alignment.append("AlignLeft")
                elif alignment[i].lower() == "c":
                    parsed_alignment.append("AlignCenter")
                elif alignment[i].lower() == "r":
                    parsed_alignment.append("AlignRight")
                else:
                    parsed_alignment.append("AlignDefault")
            except IndexError:
                for i in range(noOfColumn-len(parsed_alignment)):
                    parsed_alignment.append("AlignDefault")
        alignment = parsed_alignment
    # finalize table according to metadata
    header = body.pop(0) if has_header else None # panflute.TableRow(*[panflute.TableCell() for i in range(noOfColumn)]) # for panflute < 1.4.3
    table = panflute.Table(*body, header=header, caption=caption, width=column_width, alignment=alignment)
    return table

# We'll only run this for CodeBlock elements of class 'csv'
if __name__ == '__main__':
    panflute.toJSONFilter(panflute.yaml_filter, tag='csv', function=fenced_csv)

Overwriting ../bin/csv-tables.py


In [6]:
!chmod +x ../bin/csv-tables.py
print("JSON", "Simple", "="*80)
!pandoc test-simple.md -t json | ../bin/csv-tables.py
print("\n", "Native", "Simple", "="*80)
!pandoc --filter=../bin/csv-tables.py test-simple.md -t native
print("\n", "Markdown", "Simple", "="*80)
!pandoc --filter=../bin/csv-tables.py test-simple.md -t markdown
print("JSON", "Full", "="*80)
!pandoc test-full.md -t json | ../bin/csv-tables.py
print("\n", "Native", "Full", "="*80)
!pandoc --filter=../bin/csv-tables.py test-full.md -t native
print("\n", "Markdown", "Full", "="*80)
!pandoc --filter=../bin/csv-tables.py test-full.md -t markdown
print("JSON", "Include", "="*80)
!pandoc test-include.md -t json | ../bin/csv-tables.py
print("\n", "Native", "Include", "="*80)
!pandoc --filter=../bin/csv-tables.py test-include.md -t native
print("\n", "Markdown", "Include", "="*80)
!pandoc --filter=../bin/csv-tables.py test-include.md -t markdown

JSON Simple ================================================================================
{"pandoc-api-version":[1,17,0,4],"meta":{},"blocks":[{"t":"Table","c":[[],[{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"}],[0.22499999999999998,0.16874999999999998,0.22499999999999998,0.5812499999999999],[[{"t":"Para","c":[{"t":"Strong","c":[{"t":"Emph","c":[{"t":"Str","c":"Fruit"}]}]}]}],[{"t":"Para","c":[{"t":"Strikeout","c":[{"t":"Str","c":"Price"}]}]}],[{"t":"Para","c":[{"t":"Emph","c":[{"t":"Str","c":"Number"}]}]}],[{"t":"Para","c":[{"t":"Code","c":[["",[],[]],"Advantages"]}]}]],[[[{"t":"Para","c":[{"t":"Emph","c":[{"t":"Str","c":"Bananas"},{"t":"Subscript","c":[{"t":"Str","c":"1"}]}]}]}],[{"t":"Para","c":[{"t":"Str","c":"$1.34"}]}],[{"t":"Para","c":[{"t":"Str","c":"12"},{"t":"Subscript","c":[{"t":"Str","c":"units"}]}]}],[{"t":"Para","c":[{"t":"Str","c":"Benefits"},{"t":"Space"},{"t":"Str","c":"of"},{"t":"Space"},{"t":"Str","c":"eating"},{"t":"Space"},{